## Libraries

In [1]:
import os
import openai
from dotenv import load_dotenv
# Use load_env to trace the path of .env:
load_dotenv('.env')


True

In [2]:
openai.organization = "org-BJVQfnJYTuAJz2TkTNbchIf2"
openai.api_key = os.getenv("OPENAI_API_KEY")

In [3]:
openai.Model.list()

<OpenAIObject list at 0x7feada6bf090> JSON: {
  "data": [
    {
      "created": 1649358449,
      "id": "babbage",
      "object": "model",
      "owned_by": "openai",
      "parent": null,
      "permission": [
        {
          "allow_create_engine": false,
          "allow_fine_tuning": false,
          "allow_logprobs": true,
          "allow_sampling": true,
          "allow_search_indices": false,
          "allow_view": true,
          "created": 1669085501,
          "group": null,
          "id": "modelperm-49FUp5v084tBB49tC4z8LPH5",
          "is_blocking": false,
          "object": "model_permission",
          "organization": "*"
        }
      ],
      "root": "babbage"
    },
    {
      "created": 1649359874,
      "id": "davinci",
      "object": "model",
      "owned_by": "openai",
      "parent": null,
      "permission": [
        {
          "allow_create_engine": false,
          "allow_fine_tuning": false,
          "allow_logprobs": true,
          "allow_sa

In [4]:
import pandas as pd
from typing import Set
from transformers import GPT2TokenizerFast

import argparse, sys
import numpy as np

from PyPDF2 import PdfReader
import csv
import pickle

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import requests
from bs4 import BeautifulSoup


import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/karinabuchimartinez/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
import sys

In [7]:
from pdfminer.high_level import extract_text, extract_pages
from pdfminer.layout import LTTextContainer
from pdfminer.high_level import extract_pages
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from io import StringIO

In [8]:
!pip freeze > requirements.txt


## PDF

In [9]:
pdf_file = "Support-FAQ-clean.pdf"


In [10]:
import nltk
nltk.download('punkt')

from nltk.tokenize.punkt import PunktSentenceTokenizer
tokenizer = PunktSentenceTokenizer()

def parser(file_path):
        # Parse the PDF file
    reader = PyPDF2.PdfReader(file_path)
    # Loop over each page in the PDF document
    sentences = []
    for page in range(len(reader.pages)):
        # Extract the text from the page
        pdf_text = reader.pages[page].extract_text()
        
        # in my case, when it had '\n', I called it a new paragraph, 
        # like a collection of sentences
        paragraphs = [p for p in pdf_text.split('\n') if p]
        # and here, sent_tokenize each one of the paragraphs
        for paragraph in paragraphs:
            pdf_sentences = tokenizer.tokenize(paragraph)
    
            # Add the sentences to the list of sentences
            sentences.extend(pdf_sentences)
    return sentences

def search_question(sentences, question):

    # Search for the question in the sentences
    best_sentence = None
    best_score = 0
    for sentence in sentences:
        # Calculate the score for the sentence based on the number of overlapping words with the question
        score = len(set(re.findall(r'\b\w+\b', sentence.lower())) & set(re.findall(r'\b\w+\b', question.lower())))
        # Update the best sentence and score if this sentence has a higher score than the current best sentence
        if score > best_score:
            best_sentence = sentence
            best_score = score
    # Return the best sentence as the answer
    return best_sentence

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/karinabuchimartinez/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [11]:
def get_text_lines(pdf_file):
    """
    Gets all horizontal lines of text in a PDF
    """
    text_lines = []
    for page_layout in pdf_layout:
        for element in page_layout:
            if isinstance(element, LTTextBoxHorizontal):
                for text_line in element:
                    if isinstance(text_line, LTTextLineHorizontal):
                        text_lines.append(text_line)
    return text_lines

def unify_text_lines(text_lines):
    """
    Unifica varias líneas de texto en una sola frase
    """
    # Sort lines of text by their vertical position
    text_lines.sort(key=lambda x: -x.y0)
    
    # Concatenate the content of the lines of text
    text = ' '.join(line.get_text().strip() for line in text_lines)
    
    return text

In [12]:

tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

def count_tokens(text: str) -> int:
    """count the number of tokens in a string"""
    return len(tokenizer.encode(text))

In [13]:
def extract_page(page: str,
    index: int
) -> str:
    
    """
    Extracts the content and token count from the given page
    """
    content = ' '.join([el.get_text().strip() for el in page if isinstance(el, LTTextContainer)])
    token_count = count_tokens(content) + 4 # adding 4 extra tokens
    return ("Page " + str(index), content, token_count)

# extract the pages from the PDF file and create a list of tuples
res = []
for i, page in enumerate(extract_pages(pdf_file)):
    res.append(extract_page(page, i+1))

# create a Pandas DataFrame from the list
df = pd.DataFrame(res, columns=["title", "content", "tokens"])

# filter out pages with token count over 2046
df = df[df.tokens < 2046]

# reset the index and drop the old index column
df = df.reset_index(drop=True)

# display the first few rows of the DataFrame
print(df.head())

    title                                            content  tokens
0  Page 1  ¿Cómo se utiliza un cupón? Actualmente, los cu...     521
1  Page 2  No encuentro mi ticket Puedes acceder a tus en...     358
2  Page 3  ¿Puedo cambiar la fecha de mi ticket? Generalm...     201
3  Page 4  ¿Puedo devolver mi ticket? La mayoría de los t...     201
4  Page 5  Soy usuario, ¿cómo puedo contactarlos? Nuestro...     278


In [14]:
df

,title,content,tokens
0,Page 1,"¿Cómo se utiliza un cupón? Actualmente, los cu...",521
1,Page 2,No encuentro mi ticket Puedes acceder a tus en...,358
2,Page 3,¿Puedo cambiar la fecha de mi ticket? Generalm...,201
3,Page 4,¿Puedo devolver mi ticket? La mayoría de los t...,201
4,Page 5,"Soy usuario, ¿cómo puedo contactarlos? Nuestro...",278
5,Page 6,¿Qué es el monedero Fever Club? Fever Club es ...,521
6,Page 7,¿Cómo utilizo mi monedero Fever Club? Puedes a...,387
7,Page 8,¿Qué pasa si no encuentro mi monedero Fever en...,129
8,Page 9,¿Qué es un cupón de bienvenida? El cupón de bi...,290
9,Page 10,No encuentro mi ticket Puedes acceder a tus en...,358


In [15]:
COMPLETIONS_MODEL = "text-davinci-003"

MODEL_NAME = "curie"

DOC_EMBEDDINGS_MODEL = f"text-search-{MODEL_NAME}-doc-001"
QUERY_EMBEDDINGS_MODEL = f"text-search-{MODEL_NAME}-query-001"

def get_embedding(text: str, model: str=DOC_EMBEDDINGS_MODEL) -> list[float]:
    result = openai.Embedding.create(
      model=model,
      input=text
    )
    return result["data"][0]["embedding"]

In [16]:
def compute_doc_embeddings(df: pd.DataFrame) -> dict[tuple[str], list[float]]:
    """
    Create an embedding for each row in the dataframe using the OpenAI Embeddings API.
    Return a dictionary that maps between each embedding vector and the index of the row that it corresponds to.
    """
    embeddings_dict = {}
    for idx, r in df.iterrows():
        content = r["content"]
        embedding = get_embedding(content)
        embeddings_dict[(content, idx)] = embedding
    return embeddings_dict
    return {
        idx: get_embedding(r.content) for idx, r in df.iterrows()
    }

## Generate 

In [17]:
doc_embeddings = compute_doc_embeddings(df)

In [18]:
doc_embeddings

{('¿Cómo se utiliza un cupón? Actualmente, los cupones solo se pueden canjear en la app de Fever (puedes descargarla para Android o iOS). A continuación, te explicamos cómo utilizarlos. Para aplicar un cupón a una compra: 1. Abre la app de Fever e inicia sesión con tu método de acceso a Fever (email, Facebook, Gmail o ID de Apple). 2. Ve a la experiencia que quieras comprar, elige la fecha y hora, luego selecciona el número de tickets y haz clic en "Comprar". 3. Verás la opción "Aplicar cupón". Haz clic en ella y esto te llevará a tu lista de cupones. 4. Selecciona el cupón que desees utilizar (máximo un cupón por compra) y haz clic en "Aplicar cupón". Esto aplicará un descuento al importe total. ● Si un cupón aparece como "No disponible", significa que no se puede aplicar en esa experiencia concreta. ● Si tienes un código de cupón, haz clic en "Añadir código de cupón" para introducirlo. 5. Haz clic en "Pagar ahora" y ¡listo! Encontrarás tu(s) nuevo(s) tickets(s) en la sección "Tickets

## Save

In [19]:
with open('embeddings.pkl', 'wb') as f:
    pickle.dump(doc_embeddings, f)

## Load

In [20]:
with open('embeddings.pkl', 'rb') as f:
    doc_embeddings = pickle.load(f)

## Ask

In [21]:
from sklearn.metrics.pairwise import cosine_similarity

In [22]:
question_asked = 'Han rechazado mi cupon'

In [23]:

MAX_SECTION_LEN = 500
SEPARATOR = "\n* "
separator_len = 3

def construct_prompt(question: str, context_embeddings: dict, df: pd.DataFrame, section_index) -> tuple[str, str]:
    document_section = df.iloc[section_index]
    chosen_sections = []
    chosen_sections_len = 0
    chosen_sections_indexes = []

    chosen_sections_len += document_section.tokens + separator_len
    if chosen_sections_len > MAX_SECTION_LEN:
        space_left = MAX_SECTION_LEN - chosen_sections_len - len(SEPARATOR)
        chosen_sections.append(SEPARATOR + document_section.content[:space_left])
        chosen_sections_indexes.append(str(section_index))

    chosen_sections.append(SEPARATOR + document_section.content)
    chosen_sections_indexes.append(str(section_index))
    
    header = 'Manten tus respuestas en máximo 3 oraciones. Se conciso, y completa siempre las oraciones. \n Este es un contexto que puede ser útil :\n'
    
    return (header + "".join(chosen_sections) + "\n\n\nQ: " + question + "\n\nA: "), ("".join(chosen_sections))


    

In [24]:
COMPLETIONS_API_PARAMS = {
    # We use temperature of 0.0 because it gives the most predictable, factual answer.
    "temperature": 0.0,
    "max_tokens": 150,
    "model": COMPLETIONS_MODEL,
} 

def answer_query_with_context(query, df, embeddings):
      # Compute query embedding
    #query_embedding = np.mean(embeddings.embed_sentences([query]), axis=0)
    query_embedding = np.array(get_embedding(query))
    

    # Compute cosine similarity between query embedding and all document embeddings
    #similarities = cosine_similarity(embeddings.embedding_matrix, query_embedding.reshape(1, -1))
    similarities = cosine_similarity(list(embeddings.values()), query_embedding.reshape(1,-1))
    
    # Find index of most similar document
    most_similar_index = np.argmax(similarities)
    
    print(most_similar_index)
    
    #Construct Prompt
    prompt, context = construct_prompt(
        query,
        embeddings,
        df,
        most_similar_index
    )

    print("===\n", prompt)

    response = openai.Completion.create(
                prompt=prompt,
                **COMPLETIONS_API_PARAMS
            )

    return response["choices"][0]["text"].strip(" \n"), context

In [25]:
answer, context = answer_query_with_context(question_asked, df, doc_embeddings)

17
===
 Manten tus respuestas en máximo 3 oraciones. Se conciso, y completa siempre las oraciones. 
 Este es un contexto que puede ser útil :

* He hecho una compra y no se ha aplicado el cupón Cada cupón tiene sus propias condiciones (fecha de caducidad, limitaciones, etc.); puedes comprobarlas yendo a Perfil > Cupones y pulsando sobre el cupón. Los cupones deben aplicarse durante el proceso de compra, antes de la confirmación final del pago. Si el cupón no puede aplicarse a la experiencia elegida, no podrás seleccionarlo (lo verás en "Cupones no disponibles"). Podrás ver un desglose completo de tu compra en la página de confirmación final, incluyendo el precio de la(s) entrada(s) seleccionada(s) y el descuento aplicado (si lo hubiera). Te sugerimos que revises detenidamente esta información antes de confirmar la compra para evitar cargos imprevistos; el importe final que se muestre se cargará en el método de pago que hayas seleccionado. Para saber más sobre cómo utilizar los cupones 

In [26]:
answer

'Puede que el cupón no se aplique a la experiencia elegida, o que haya caducado. Por favor, revisa las condiciones del cupón en Perfil > Cupones y asegúrate de que se aplica a la compra antes de confirmarla. Si tienes alguna duda, puedes consultar nuestra guía sobre cómo utilizar los cupones.'

In [27]:
answer, context = answer_query_with_context('Como usar los cupones', df, doc_embeddings)
print(answer)

16
===
 Manten tus respuestas en máximo 3 oraciones. Se conciso, y completa siempre las oraciones. 
 Este es un contexto que puede ser útil :

* Acerca de la fecha de caducidad de los cupones Cada cupón tiene sus propias condiciones (fecha de caducidad, limitaciones, etc.); puedes comprobarlas dirigiéndote a la sección Cupones de la aplicación Fever y pulsando sobre el cupón. Ten en cuenta que una vez caducado, el cupón desaparecerá y no podrás utilizarlo. Asegúrate de comprobar la fecha de caducidad con antelación.


Q: Como usar los cupones

A: 
Puedes utilizar los cupones dirigiéndote a la sección Cupones de la aplicación Fever y pulsando sobre el cupón. Asegúrate de comprobar la fecha de caducidad con antelación para evitar que caduque.


In [28]:
answer, context = answer_query_with_context('No pude ', df, doc_embeddings)
print(answer)

11
===
 Manten tus respuestas en máximo 3 oraciones. Se conciso, y completa siempre las oraciones. 
 Este es un contexto que puede ser útil :

* Pago rechazado: Si tu pago ha sido rechazado, comprueba que tu tarjeta de crédito no está caducada y que dispone de fondos suficientes para realizar la transacción. Si el problema persiste, te sugerimos que utilices otro método de pago o que te pongas en contacto directamente con tu banco, ya que este no nos facilita ninguna información sobre el motivo del rechazo del pago para proteger tu seguridad y privacidad.


Q: No pude 

A: 
Intenta utilizar otro método de pago o contacta a tu banco para obtener más información.


## Streamlit 

In [34]:
import streamlit as st

In [35]:
@st.cache_data
def load_data():
    """ Utility function for loading the penguins dataset as a dataframe."""
    # df = sns.load_dataset('penguins')

    with open('embeddings.pkl', 'rb') as f:
        doc_embeddings = pickle.load(f)
    df = pd.read_csv('paginas.txt')
    return df, doc_embeddings

# load dataset
df, doc_embeddings = load_data()



    


In [ ]:
# Ask a question and search for the answer
question = st.text_input("Ask a question:")
if question:
    answer, context = answer_query_with_context(question, df, doc_embeddings)
    # Replace newline characters with line breaks
    answer_with_line_breaks = answer.replace('\n', '<br>')
    # Display the answer as a paragraph of text with line breaks
    st.markdown(f"<p>{answer_with_line_breaks}</p>", unsafe_allow_html=True)
    st.markdown("""
    <style>
    p {
        font-size: 18px;
        line-height: 1.5;
        text-align: justify;
        }